## Import Tools

In [22]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
import time
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

## Dataset Processing

### Read in the original dataset

In [ ]:
train_dl_origin = torch.load('Dataset/train_dl.pt', weights_only=False) 
valid_dl_origin = torch.load('Dataset/valid_dl.pt', weights_only=False)

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

### CSI Processing: Take Modulus of complex matrices

In [24]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [25]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


### CSI Processing: Normalize to [0,1]

In [ ]:
# using training set statistics for both datasets to prevent data leakage. If instead used valid csi modulus - valid csi mod min, this would not accurately validate model that was trained and let model "cheat"
train_min = train_CSI_modulus.min()
train_max = train_CSI_modulus.max()

train_CSI_norm = (train_CSI_modulus - train_min) / (train_max - train_min) #min max normalization: norm_value = (value - min) / (max - min)
valid_CSI_norm = (valid_CSI_modulus - train_min) / (train_max - train_min)

print(train_CSI_norm.shape)
print(valid_CSI_norm.shape)
print(train_label.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
torch.Size([15000])


### Preparing Data for ML

In [ ]:
# flattening data to be 2D
train_CSI_ML = train_CSI_norm.reshape(train_CSI_norm.shape[0], -1) # calculating 15000 x (1 x 4 x 1632), to 1d vector
valid_CSI_ML = valid_CSI_norm.reshape(valid_CSI_norm.shape[0], -1)

print(train_CSI_ML.shape)

torch.Size([15000, 6528])


## Machine Learning: Logistic Regression

In [ ]:
# creating and initializaing logistic regression model

randstate = 31
logistic_regression = LogisticRegression(max_iter=1000, random_state=randstate)

start_time = time.time() # recording start time

logistic_regression.fit(train_CSI_ML, train_label) #training 

train_time = time.time() - start_time # training time

# Evaluation and performance on training set

train_pred = logistic_regression.predict(train_CSI_ML)      # model performance/outputs on its own training data after fit
train_acc = accuracy_score(train_label, train_pred)         # training accuracy compared to labels
train_precision = precision_score(train_label, train_pred)  # precision score
train_recall = recall_score(train_label, train_pred)        # recall
train_f1 = f1_score(train_label, train_pred)                # f1 score

# Evaluating on validation set

start_valid_time = time.time()                                  # start recording time

valid_pred = logistic_regression.predict(valid_CSI_ML)          # model prediction

valid_time = time.time() - start_valid_time                     # validation time 

valid_acc = accuracy_score(valid_label, valid_pred)             # validation accuracy

valid_precision = precision_score(valid_label, valid_pred)      # precision 
valid_recall = recall_score(valid_label, valid_pred)            # recall
valid_f1 = f1_score(valid_label, valid_pred)                    # f1 score



print('Training Time: ', train_time)
print('Training Accuracy: ', train_acc)
print()
print('Training Precision: ', train_precision)
print('Training Recall: ', train_recall)
print('Training F1 Score: ', train_f1)
print()
print('Validation Time: ', valid_time)
print('Validation Accuracy: ', valid_acc)
print()
print('Validation Precision: ', valid_precision)
print('Validation Recall: ', valid_recall)
print('Validation F1 Score: ', valid_f1)


Training Time:  3.661696195602417
Training Accuracy:  0.9553333333333334

Training Precision:  0.9839494163424124
Training Recall:  0.7605263157894737
Training F1 Score:  0.857930449533503

Validation Time:  0.0586850643157959
Validation Accuracy:  0.9438

Validation Precision:  0.96987087517934
Validation Recall:  0.7222222222222222
Validation F1 Score:  0.8279240661359462


### Prepare Data for CNN

In [29]:
# Keeping 4D shape for CNN
train_CSI_CNN = train_CSI_norm
valid_CSI_CNN = valid_CSI_norm

print('CNN shape:', train_CSI_CNN.shape)

CNN shape: torch.Size([15000, 1, 4, 1632])


## Neural Network: CNN Classifier

In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__() # Conv2d(in_channels, out_channels, kernel_size, stride, padding) 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) # in channel = 1 (like grayscale), out channel = 32 (32 filters); filter being applied to 4 rows 1632 cols 
        self.bn1 = nn.BatchNorm2d(32) # normalization to prevent distribution issues
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # pooling reduces dimension
        self.fc1 = nn.Linear(64 * 1 * 408, 256) # fully connected layers
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.relu = nn.ReLU() # nonlinearity ReLU
        self.dropout = nn.Dropout(0.3) # prevent overfitting

    def forward(self, x):
        # Convolutional block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Convolutional block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x


# [32, 1, 4, 1632]     
# [32, 32, 4, 1632]    
# [32, 32, 2, 816]     
# [32, 64, 2, 816]     
# [32, 64, 1, 408]     
# [32, 26112]          
# [32, 256]            
# [32, 128]           
# [32, 2]            



model = CNNClassifier()

# Total parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters: {total_params:,}')

Total parameters: 6,737,090


In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss() #[32, 2] -> [[LoS_score, NLoS_score], ...] for 32 samples
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train and valid datasets
train_dataset = TensorDataset(train_CSI_CNN, train_label)
valid_dataset = TensorDataset(valid_CSI_CNN, valid_label)

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 10
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1) # Grab predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

train_time = time.time() - start_time
print(f'Training completed in {train_time:.2f} seconds')

Epoch [1/10], Loss: 0.4362, Accuracy: 82.01%
Epoch [2/10], Loss: 0.2500, Accuracy: 89.76%
Epoch [3/10], Loss: 0.1369, Accuracy: 94.44%
Epoch [4/10], Loss: 0.0932, Accuracy: 96.62%
Epoch [5/10], Loss: 0.0705, Accuracy: 97.50%
Epoch [6/10], Loss: 0.0743, Accuracy: 97.60%
Epoch [7/10], Loss: 0.0502, Accuracy: 98.27%
Epoch [8/10], Loss: 0.0358, Accuracy: 98.63%
Epoch [9/10], Loss: 0.0447, Accuracy: 98.64%
Epoch [10/10], Loss: 0.0526, Accuracy: 98.51%
Training completed in 608.08 seconds


In [32]:
# Evaluation
model.eval()
start_time = time.time()

valid_preds = []
valid_labels_list = []

with torch.no_grad():
    for inputs, labels in valid_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        valid_preds.extend(predicted.tolist())
        valid_labels_list.extend(labels.tolist())

test_time = time.time() - start_time

# Calculate metrics
cm = confusion_matrix(valid_labels_list, valid_preds)
accuracy = accuracy_score(valid_labels_list, valid_preds)
precision = precision_score(valid_labels_list, valid_preds)
recall = recall_score(valid_labels_list, valid_preds)
f1 = f1_score(valid_labels_list, valid_preds)

print('Confusion Matrix:')
print(cm)
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Testing time: {test_time:.4f}s')

Confusion Matrix:
[[4064    0]
 [  16  920]]
Accuracy: 0.9968
Precision: 1.0000
Recall: 0.9829
F1 Score: 0.9914
Testing time: 9.0245s
